In [45]:
# imports
import pandas as pd
import requests
import csv
import time
from datetime import datetime, timedelta
import zipfile
import io

In [46]:
year = 2022
node = "PALOVRDE_5_N101"

In [47]:
month_dict = {'Jan': ['0101', '0201'],
              'Feb': ['0201', '0301'],
              'Mar': ['0301', '0401'],
              'Apr': ['0401', '0501'],
              'May': ['0501', '0601'],
              'Jun': ['0601', '0701'],
              'Jul': ['0701', '0801'],
              'Aug': ['0801', '0901'],
              'Sep': ['0901', '1001'],
              'Oct': ['1001', '1101'],
              'Nov': ['1101', '1201'],
              'Dec': ['1201', '1231']
              }
dataframes = []
string1= 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime='
string2= 'T08:00-0000&enddatetime='
string3= 'T08:00-0000&market_run_id=DAM&node='


In [48]:
links = [f"{string1}{year}{month_dict[month][0]}{string2}{year}{month_dict[month][1]}{string3}{node}" for month in month_dict.keys()]

In [49]:
links

['http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220101T08:00-0000&enddatetime=20220201T08:00-0000&market_run_id=DAM&node=PALOVRDE_5_N101',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220201T08:00-0000&enddatetime=20220301T08:00-0000&market_run_id=DAM&node=PALOVRDE_5_N101',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220301T08:00-0000&enddatetime=20220401T08:00-0000&market_run_id=DAM&node=PALOVRDE_5_N101',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220401T08:00-0000&enddatetime=20220501T08:00-0000&market_run_id=DAM&node=PALOVRDE_5_N101',
 'http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&startdatetime=20220501T08:00-0000&enddatetime=20220601T08:00-0000&market_run_id=DAM&node=PALOVRDE_5_N101',
 'http://oasis.caiso.com/

In [50]:
# Creates a CSV file of the input links for future reference
with open('links.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for link in links:
        writer.writerow([link])

In [32]:
# Define the list of links to ZIP files
# Initialize an empty list to store the data from the CSV files
data = []

# Loop through the ZIP links and retrieve the data from each CSV file
for link in links:
    response = requests.get(link)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    for file in zip_file.namelist():
        if file.endswith('.csv'):
            content = zip_file.read(file)
            decoded_content = content.decode('utf-8')
            csvreader = csv.reader(decoded_content.splitlines(), delimiter=',')
            for row in csvreader:
                data.append(row)
    time.sleep(5)

# Write the data to a new CSV file
with open('unfiltered.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    for row in data:
        writer.writerow(row)

In [36]:
# Open the input CSV file and read the rows
with open('unfiltered.csv', 'r', newline='', encoding='utf-8') as infile:
    reader = csv.reader(infile)

    # Get the index of the "LMP_TYPE", "OPR_DT", and "OPR_HR" columns
    header = next(reader)
    lmp_type_index = header.index("LMP_TYPE")
    opr_dt_index = header.index("OPR_DT")
    opr_hr_index = header.index("OPR_HR")

    # Filter the rows to only include those where the value in the "LMP_TYPE" column is "LMP"
    filtered_rows = []
    for row in reader:
        if row[lmp_type_index] == "LMP":
            filtered_rows.append(row)

    # Convert the "OPR_HR" column values to integers and sort the filtered rows by "OPR_DT" and then by "OPR_HR"
    sorted_rows = sorted(filtered_rows, key=lambda x: (x[opr_dt_index], int(x[opr_hr_index])))

# Write the sorted rows to a new CSV file
with open('filtered_sorted.csv', 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(header)
    writer.writerows(sorted_rows)